In [1]:
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter 
from matplotlib.colors import ListedColormap
import string
import re
from scipy.stats import hmean
from scipy.stats import norm
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline
pth = r"C:\EPFL\2018-2019\nltk_data" #change location according to your nltk data path
nltk.data.path.append(pth)
sns.set()

In [2]:
import sklearn
#from sklearn.model_selection.cross_validation import train_test_split
from sklearn import metrics, model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score

In [4]:
from sklearn.model_selection import GridSearchCV

**Description of a pipeline. Used to apply sequence of transform. I had issue with cvec, needed to apply it two times on x_train and x_test, it gave bad results...**

https://medium.com/@chrisfotache/text-classification-in-python-pipelines-nlp-nltk-tf-idf-xgboost-and-more-b83451a327e0
https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

https://medium.com/@chrisfotache/text-classification-in-python-pipelines-nlp-nltk-tf-idf-xgboost-and-more-b83451a327e0
https://stackoverflow.com/questions/43366561/use-sklearns-gridsearchcv-with-a-pipeline-preprocessing-just-once

https://www.kaggle.com/cesartrevisan/scikit-learn-and-gridsearchcv

# Multi Classifier Comparison

**The best results we got so far were with tf-idf (even if the difference was not that significant with Countvectorizer) along with 100 000 features and use of unigram and bigram. Now we will compare using the same pipeline other models. Due to the expensive computation time, we will compare these models with a maximum of 10 000 features only and using search grid to get the best estimator each time**

In [16]:
from time import time
import multiprocessing
import cython

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC #support vector machine SVM
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.feature_selection import SelectFromModel

In [9]:
data = pd.read_pickle('clean_tweets')
X = data.text.values
y = data.sentiment.values
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
names = ["Logistic Regression", "Linear SVC", "Random Forest","Ridge Classifier"]

classifiers = [LogisticRegression(),LinearSVC(), RandomForestClassifier(), RidgeClassifier()]
zipped_clf = zip(names,classifiers)

In [11]:
tvec = TfidfVectorizer()

In [17]:
%load_ext cythonmagic
def classifier_comparator(vectorizer, ngram_range, classifier_list, n_features=10000, stop_words=None):
    result = []
    vectorizer.set_params(stop_words=stop_words, max_features=n_features, ngram_range=ngram_range)
    for n,c in classifier_list:
        checker_pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', c)
        ])
        
        #----------- SEARCH GRID -----------
        print("-----------------------------------------------------------------")
        print("Choosen classifier : {}".format(n))
        t0 = time()
        if(n == "Logistic Regression") :
            hyperparameters = dict(C=np.logspace(0, 4, 10), penalty=['l1', 'l2'])
            clf = GridSearchCV(c, hyperparameters, cv=5, verbose=0)
        elif(n == "Random Forest") :
            hyperparameters = {"max_depth": [3, None]}
        elif(n == "Ridge Classifier") :
            hyperparameters = dict(alpha = np.array([0.01,0.001,0.0001]))
        elif(n == "Linear SVC") :
            Cs = [0.01, 0.1, 1]
            hyperparameters = {'C': Cs}
        
        #search for best estimator
        clf = GridSearchCV(c, hyperparameters, cv=5, verbose=0)
        vectorizer.set_params(max_features=n_features, ngram_range=ngram_range)
        best_estimator = Pipeline([('vectorizer', vectorizer),('classifier', clf)])
        #best_estimator = Pipeline([('vectorizer', vectorizer),('classifier', c)])

        #---------- predict with best model ------------
        classifier_fit = best_estimator.fit(X_train, y_train)
        y_pred = classifier_fit.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        tt_time = time() - t0
        result.append([n, accuracy, tt_time])
        print("Accuracy score is {} :".format(accuracy))
        print("Train and test time took {}".format(tt_time))
    return result

The cythonmagic extension is already loaded. To reload it, use:
  %reload_ext cythonmagic


In [18]:
%%time
#pool = multiprocessing.Pool(processes=2)
#r = pool.map(classifier_comparator, years)
#pool.close()
all_results = classifier_comparator(tvec, range(1, 3), zipped_clf, n_features=10000)

-----------------------------------------------------------------
Choosen classifier : Logistic Regression
Accuracy score is 0.817795856107792 :
Train and test time took 605.3928697109222
-----------------------------------------------------------------
Choosen classifier : Linear SVC
Accuracy score is 0.8174145163340536 :
Train and test time took 40.70728349685669
-----------------------------------------------------------------
Choosen classifier : Random Forest
Accuracy score is 0.7755688318291598 :
Train and test time took 321.2097415924072
-----------------------------------------------------------------
Choosen classifier : Ridge Classifier
Accuracy score is 0.8126859031396975 :
Train and test time took 461.7082438468933
Wall time: 23min 49s
